In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from utils import get_network, get_training_dataloader, get_test_dataloader, WarmUpLR, \
    most_recent_folder, most_recent_weights, last_epoch, best_acc_weights, get_student_network
from conf import settings
from models.vgg import vgg16_bn
from torch.profiler import profile, record_function, ProfilerActivity
import copy
from torchinfo import summary

In [3]:
!pip install gdown

     |████████████████████████████████| 77 kB 4.0 MB/s eta 0:00:01


In [13]:
!gdown "https://drive.google.com/u/0/uc?id=1nxJm5Eug45hYxGNPbQ1rTjHMJBN1asXV&export=download"

Downloading...
From (uriginal): https://drive.google.com/u/0/uc?id=1nxJm5Eug45hYxGNPbQ1rTjHMJBN1asXV&export=download
From (redirected): https://drive.google.com/uc?id=1nxJm5Eug45hYxGNPbQ1rTjHMJBN1asXV&export=download&confirm=t&uuid=79bd94d6-da1c-4351-8771-78503da2effb
To: /home/studio-lab-user/sagemaker-studiolab-notebooks/pytorch-cifar100/v5-200-regular.pth
100%|██████████████████████████████████████| 98.6M/98.6M [00:02<00:00, 33.0MB/s]


In [6]:
net = get_student_network("v2")

In [8]:
from torchinfo import summary
summary(net, input_size = (1,3,32,32))

Layer (type:depth-idx)                   Output Shape              Param #
MyCompressNet2                           [1, 100]                  --
├─Conv2d: 1-1                            [1, 64, 32, 32]           1,792
├─BatchNorm2d: 1-2                       [1, 64, 32, 32]           128
├─Conv2d: 1-3                            [1, 128, 16, 16]          73,856
├─BatchNorm2d: 1-4                       [1, 128, 16, 16]          256
├─Conv2d: 1-5                            [1, 128, 16, 16]          147,584
├─BatchNorm2d: 1-6                       [1, 128, 16, 16]          256
├─Conv2d: 1-7                            [1, 256, 8, 8]            295,168
├─BatchNorm2d: 1-8                       [1, 256, 8, 8]            512
├─Conv2d: 1-9                            [1, 256, 8, 8]            590,080
├─BatchNorm2d: 1-10                      [1, 256, 8, 8]            512
├─Conv2d: 1-11                           [1, 256, 4, 4]            590,080
├─BatchNorm2d: 1-12                      [1, 256, 4, 

In [16]:
!python3 test.py -net=v3 -weights=v3-200-regular.pth -b=128

Files already downloaded and verified
MyCompressNet3(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), padding=same)
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), padding=same)
  (bn5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn6): BatchNorm2d(256, eps=1

In [15]:
!pip install matplotlib

  Using cached matplotlib-3.7.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.6 MB)
  Using cached kiwisolver-1.4.4-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.6 MB)
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)
  Using cached contourpy-1.1.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (300 kB)
  Using cached fonttools-4.41.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.5 MB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)


In [2]:
class Fire(nn.Module):
    def __init__(self, inplanes: int, squeeze_planes: int, expand1x1_planes: int, expand3x3_planes: int) -> None:
        super().__init__()
        self.inplanes = inplanes
        self.squeeze = nn.Conv2d(inplanes, squeeze_planes, kernel_size=1)
        self.squeeze_activation = nn.ReLU(inplace=True)
        self.expand1x1 = nn.Conv2d(squeeze_planes, expand1x1_planes, kernel_size=1)
        self.expand1x1_activation = nn.ReLU(inplace=True)
        self.expand3x3 = nn.Conv2d(squeeze_planes, expand3x3_planes, kernel_size=3, padding=1)
        self.expand3x3_activation = nn.ReLU(inplace=True)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.squeeze_activation(self.squeeze(x))
        return torch.cat(
            [self.expand1x1_activation(self.expand1x1(x)), self.expand3x3_activation(self.expand3x3(x))], 1
        )

In [3]:
model = Fire(3,32,64,128)

In [5]:
model.forward(torch.rand(1,3,70,70)).shape

torch.Size([1, 192, 70, 70])

In [19]:
class MyCompressNet2(nn.Module):
    def __init__(self, num_channels = 64, dr_rate = 0.3):
        super(MyCompressNet2, self).__init__()
        self.num_channels = num_channels
        self.conv1 = nn.Conv2d(3, self.num_channels, 3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(self.num_channels)
        self.conv2 = nn.Conv2d(self.num_channels, self.num_channels*2, 3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(self.num_channels*2)
        self.fire1 = Fire(self.num_channels*2, self.num_channels*4, self.num_channels*2, self.num_channels*2)
        self.bnfire1 = nn.BatchNorm2d(self.num_channels*4)
        self.conv3 = nn.Conv2d(self.num_channels*4, self.num_channels*4, 3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(self.num_channels*4)
        # self.conv4 = nn.Conv2d(self.num_channels*2, self.num_channels*4, 3, stride=1, padding=1)
        self.fire2 = Fire(self.num_channels*4, self.num_channels*8, self.num_channels*4, self.num_channels*4)
        self.bnfire2 = nn.BatchNorm2d(self.num_channels*8)
        self.conv5 = nn.Conv2d(self.num_channels*8, self.num_channels*4, 3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(self.num_channels*4)
        self.fc1 = nn.Linear(4*4*self.num_channels*4, self.num_channels*4*4)
        self.fc2 = nn.Linear(self.num_channels*4*4, self.num_channels*4*4)
        self.fc3 = nn.Linear(self.num_channels*4*4, self.num_channels*4)
        self.fc4 = nn.Linear(self.num_channels*4, self.num_channels*4)
        self.fc5 = nn.Linear(self.num_channels*4, self.num_channels*2)
        self.fc6 = nn.Linear(self.num_channels*2, 100)      
        self.dropout_rate = dr_rate
    def forward(self,x):
        """
        Forward function
        """
        x = self.bn1(self.conv1(x))
        x = F.relu(F.max_pool2d(x,2))
        x = self.bn2(self.conv2(x))
        x = F.relu(x)
        x = F.relu(self.bnfire1(self.fire1(x)))
        x = self.bn3(self.conv3(x))
        x = F.relu(F.max_pool2d(x,2))
        # x = self.bn4(self.conv4(x))
        # x = F.relu(x)
        x = self.bnfire2(self.fire2(x))
        x = F.relu(x)
        x = self.bn5(self.conv5(x))
        x = F.relu(F.max_pool2d(x,2))
        x = x.view(-1, 4*4*self.num_channels*4)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p = self.dropout_rate)
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.dropout(x, p = self.dropout_rate)
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return x
        


In [24]:
student_net = MyCompressNet2(64, 0.3)

In [25]:
summary(student_net, input_size= (1,3,32,32))

Layer (type:depth-idx)                   Output Shape              Param #
MyCompressNet2                           [1, 100]                  --
├─Conv2d: 1-1                            [1, 64, 32, 32]           1,792
├─BatchNorm2d: 1-2                       [1, 64, 32, 32]           128
├─Conv2d: 1-3                            [1, 128, 16, 16]          73,856
├─BatchNorm2d: 1-4                       [1, 128, 16, 16]          256
├─Fire: 1-5                              [1, 256, 16, 16]          --
│    └─Conv2d: 2-1                       [1, 256, 16, 16]          33,024
│    └─ReLU: 2-2                         [1, 256, 16, 16]          --
│    └─Conv2d: 2-3                       [1, 128, 16, 16]          32,896
│    └─ReLU: 2-4                         [1, 128, 16, 16]          --
│    └─Conv2d: 2-5                       [1, 128, 16, 16]          295,040
│    └─ReLU: 2-6                         [1, 128, 16, 16]          --
├─BatchNorm2d: 1-6                       [1, 256, 16, 16]      

In [23]:
student_net.forward(torch.rand(1,3,32,32)).shape

torch.Size([1, 100])